In [135]:
import gc
import multiprocessing
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.model_selection import KFold
from sklearn.feature_selection import RFECV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import seaborn as sns
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_fscore_support as prfs, accuracy_score
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score, precision_score
from sklearn.utils import resample

In [136]:
df = pd.read_csv("../curated-data/CorrelationData_conf_SR3.csv")

In [137]:
df.SR.value_counts()

2    136
1     31
Name: SR, dtype: int64

In [138]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df, df['SR']):
    strat_train_set = df.loc[train_index]
    strat_test_set = df.loc[test_index]

X_train = strat_train_set.drop('SR', axis=1)
Y_train = strat_train_set['SR'].copy()

X_test = strat_test_set.drop('SR', axis=1)
Y_test = strat_test_set['SR'].copy()

In [139]:
X = pd.concat([X_train, Y_train], axis=1)
X.info()
X.SR.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 13 to 156
Data columns (total 26 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rank    133 non-null    int64  
 1   NASA    133 non-null    int64  
 2   TA      133 non-null    int64  
 3   EXT     133 non-null    int64  
 4   AGR     133 non-null    int64  
 5   CS      133 non-null    int64  
 6   NT      133 non-null    int64  
 7   OP      133 non-null    int64  
 8   AV      133 non-null    int64  
 9   EM      133 non-null    int64  
 10  Task    133 non-null    int64  
 11  H       133 non-null    float64
 12  RS      133 non-null    int64  
 13  WH      133 non-null    int64  
 14  TWR     133 non-null    int64  
 15  BR      133 non-null    int64  
 16  NCP     133 non-null    int64  
 17  CR      133 non-null    int64  
 18  ACP     133 non-null    int64  
 19  AR      133 non-null    int64  
 20  DWH     133 non-null    int64  
 21  DWR     133 non-null    int64  
 22  D

2    108
1     25
Name: SR, dtype: int64

In [153]:
df.SR.value_counts()

2    136
1     31
Name: SR, dtype: int64

In [140]:
Success = X[X.SR==2]
Not_Success = X[X.SR==1]

# upsample minority
N_success_upsampled = resample(Not_Success,
                          replace=True, # sample with replacement
                          n_samples=len(Success), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([Success, N_success_upsampled])

# check new class counts
upsampled.SR.value_counts()

2    108
1    108
Name: SR, dtype: int64

In [141]:
Y_train = upsampled.SR
X_train = upsampled.drop('SR', axis=1)

In [142]:
rf = RandomForestClassifier(n_estimators = 1000, bootstrap = True)
rf.fit(X_train, Y_train)

y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)
pd.DataFrame(confusion_matrix(Y_test, y_pred))

,0,1
0,1,5
1,1,27


In [143]:
print("Accuracy: ", accuracy_score(Y_test, y_pred))
print("Precision: ", precision_score(Y_test, y_pred))
print("Recall: ", recall_score(Y_test, y_pred))
print("F1: ", f1_score(Y_test, y_pred))
print("Auc Score: ", roc_auc_score(Y_test, y_score[:,1]))

Accuracy:  0.8235294117647058
Precision:  0.5
Recall:  0.16666666666666666
F1:  0.25
Auc Score:  0.6785714285714286


In [144]:
df_buffered = df[['NCP', 'EXT', 'CR', 'H', 'DWR', 'DDR', 'Rank', 'DWH', 'T', 'DS', 'SR']].copy()
df_buffered

,NCP,EXT,CR,H,DWR,DDR,Rank,DWH,T,DS,SR
0,2,2,1,3.713596,50,90,3,3,2,1,2
1,1,6,2,3.218916,30,30,2,1,3,1,2
2,3,6,2,4.094361,50,50,3,1,5,1,2
3,2,5,2,3.401231,20,20,3,1,5,1,2
4,1,6,2,2.772651,50,30,2,1,4,1,2
...,...,...,...,...,...,...,...,...,...,...,...
162,2,7,2,4.174403,70,70,3,1,5,1,1
163,2,6,2,1.386544,0,0,1,1,1,1,1
164,2,8,3,3.178095,70,50,3,2,3,2,2
165,2,6,2,3.135538,70,70,2,1,2,1,2


In [145]:
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(df_buffered, df_buffered['SR']):
    strat_train_set = df_buffered.loc[train_index]
    strat_test_set = df_buffered.loc[test_index]

X_train = strat_train_set.drop('SR', axis=1)
Y_train = strat_train_set['SR'].copy()

X_test = strat_test_set.drop('SR', axis=1)
Y_test = strat_test_set['SR'].copy()

In [146]:
X = pd.concat([X_train, Y_train], axis=1)
X.info()
X.SR.value_counts()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133 entries, 13 to 156
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   NCP     133 non-null    int64  
 1   EXT     133 non-null    int64  
 2   CR      133 non-null    int64  
 3   H       133 non-null    float64
 4   DWR     133 non-null    int64  
 5   DDR     133 non-null    int64  
 6   Rank    133 non-null    int64  
 7   DWH     133 non-null    int64  
 8   T       133 non-null    int64  
 9   DS      133 non-null    int64  
 10  SR      133 non-null    int64  
dtypes: float64(1), int64(10)
memory usage: 12.5 KB


2    108
1     25
Name: SR, dtype: int64

In [147]:
Success = X[X.SR==2]
Not_Success = X[X.SR==1]

# upsample minority
N_success_upsampled = resample(Not_Success,
                          replace=True, # sample with replacement
                          n_samples=len(Success), # match number in majority class
                          random_state=27) # reproducible results

# combine majority and upsampled minority
upsampled_buffered = pd.concat([Success, N_success_upsampled])

# check new class counts
upsampled_buffered.SR.value_counts()

2    108
1    108
Name: SR, dtype: int64

In [148]:
Y_train = upsampled_buffered.SR
X_train = upsampled_buffered.drop('SR', axis=1)

In [149]:
X_train

,NCP,EXT,CR,H,DWR,DDR,Rank,DWH,T,DS
137,3,6,3,2.708117,30,30,1,1,5,2
9,1,5,1,2.397986,80,70,1,2,2,1
31,3,8,3,1.946053,70,100,1,3,2,3
124,1,10,2,2.833272,40,50,2,3,3,1
28,1,4,3,3.850169,60,60,3,1,4,1
...,...,...,...,...,...,...,...,...,...,...
13,3,7,3,3.761223,70,80,3,1,2,1
106,3,6,2,2.197336,50,40,2,2,1,2
115,1,7,2,2.197336,70,100,1,2,3,1
37,1,2,2,1.098946,100,90,2,3,5,3


In [150]:
rf = RandomForestClassifier(n_estimators = 2000, bootstrap = True)
rf.fit(X_train, Y_train)

y_pred = rf.predict(X_test)
y_score = rf.predict_proba(X_test)
pd.DataFrame(confusion_matrix(Y_test, y_pred))

,0,1
0,2,4
1,4,24


In [151]:
print("Accuracy: ", accuracy_score(Y_test, y_pred))
print("Precision: ", precision_score(Y_test, y_pred))
print("Recall: ", recall_score(Y_test, y_pred))
print("F1: ", f1_score(Y_test, y_pred))
print("Auc Score: ", roc_auc_score(Y_test, y_score[:,1]))

Accuracy:  0.7647058823529411
Precision:  0.3333333333333333
Recall:  0.3333333333333333
F1:  0.3333333333333333
Auc Score:  0.7083333333333334
